In [7]:
from matplotlib import pyplot as plt
# from brokenaxes import brokenaxes
import numpy as np
import os
import sys
import matplotlib.ticker as mticker
from matplotlib.ticker import FuncFormatter

my_figsize = [10, 8]
my_fontsize = 35
WIDTH = 0.05  # the width of the bars
SHORT = 100000 # 100KB
LONG = 10000000 # 1000KB
LOADS = ['0.1','0.25', '0.5', '0.75','1']
labels = ['10','25', '50', '75','100'] # LOADS

# LOADS = ['1']
# labels = ['100']
# LOADS = [2, 4, 6, 8, 10, 12, 14, 16]
# labels = [2, 4, 6, 8, 10, 12, 14, 16]
# LOADS = [2, 4, 6, 8, 10, 15, 20, 30, 40, 50]
# labels = [2, 4, 6, 8, 10, 15, 20, 30, 40, 50]

# LOADS = ['0.25']
# labels = ['2', '3', '4', '5', '6']
# WORKLOADS = ['W1', 'W2', 'W3', 'W4']
WORKLOADS = ['W4_0.1', 'W4_0.25', 'W4_0.5', 'W4_0.75', 'W4_1']
# WORKLOADS = ['incast']
# WORKLOADS = ['uniformincast']


METRICS = ['FCT', 'GOODPUT']


os.chdir(sys.path[0])
os.chdir('./')

msses = [128, 256, 512, 1024]
bppns = [20, 40, 80, 160, 320, 640] 


DATA_DIR_TEMP = '../DATA/Zeropod/W4/new_schedule3/mss_{mss}_bppn_{bppn}/DATA_zeropod_{workload}/'

FIG_FILE_NAME = WORKLOADS[0]
FIGURE_DIR = '../FIGS/'+ FIG_FILE_NAME +'/GOODPUT/'
if not os.path.exists(FIGURE_DIR):
    os.makedirs(FIGURE_DIR)

data = {}

for mss in msses:
    data[mss] = {}
    for bppn in bppns:
        data[mss][bppn] = {}
        for workload in WORKLOADS:
            data[mss][bppn][workload] = {}
            
for mss in msses:
    for bppn in bppns:
        for workload in WORKLOADS:
            dir_path = DATA_DIR_TEMP.format(mss=mss, bppn=bppn, workload=workload)
            data[mss][bppn][workload]['PATH'] = dir_path
            for metric in METRICS:
                data_path = dir_path + metric + '.txt'
                data[mss][bppn][workload][metric] = np.loadtxt(data_path)
                print("LOADED: ", mss, bppn, workload, metric)
                    
for mss in msses:
    for bppn in bppns:
        for workload in WORKLOADS:
                for metric in METRICS:
                    data[mss][bppn][workload]['SHORT_IDX'] = data[mss][bppn][workload]['FCT'][:, 4] < SHORT
                    data[mss][bppn][workload]['LONG_IDX'] = data[mss][bppn][workload]['FCT'][:, 4] > LONG
                    data[mss][bppn][workload]['MIDDLE_IDX'] = (data[mss][bppn][workload]['FCT'][:, 4] >= SHORT) * (data[mss][bppn][workload]['FCT'][:, 4] <= LONG)

LOADED:  128 20 W4_0.5 FCT
LOADED:  128 20 W4_0.5 GOODPUT
LOADED:  128 40 W4_0.5 FCT
LOADED:  128 40 W4_0.5 GOODPUT
LOADED:  128 80 W4_0.5 FCT
LOADED:  128 80 W4_0.5 GOODPUT
LOADED:  128 160 W4_0.5 FCT
LOADED:  128 160 W4_0.5 GOODPUT
LOADED:  128 320 W4_0.5 FCT
LOADED:  128 320 W4_0.5 GOODPUT
LOADED:  128 640 W4_0.5 FCT
LOADED:  128 640 W4_0.5 GOODPUT
LOADED:  256 20 W4_0.5 FCT
LOADED:  256 20 W4_0.5 GOODPUT
LOADED:  256 40 W4_0.5 FCT
LOADED:  256 40 W4_0.5 GOODPUT
LOADED:  256 80 W4_0.5 FCT
LOADED:  256 80 W4_0.5 GOODPUT
LOADED:  256 160 W4_0.5 FCT
LOADED:  256 160 W4_0.5 GOODPUT
LOADED:  256 320 W4_0.5 FCT
LOADED:  256 320 W4_0.5 GOODPUT
LOADED:  256 640 W4_0.5 FCT
LOADED:  256 640 W4_0.5 GOODPUT
LOADED:  512 20 W4_0.5 FCT
LOADED:  512 20 W4_0.5 GOODPUT
LOADED:  512 40 W4_0.5 FCT
LOADED:  512 40 W4_0.5 GOODPUT
LOADED:  512 80 W4_0.5 FCT
LOADED:  512 80 W4_0.5 GOODPUT
LOADED:  512 160 W4_0.5 FCT
LOADED:  512 160 W4_0.5 GOODPUT
LOADED:  512 320 W4_0.5 FCT
LOADED:  512 320 W4_0.5 GOODPU

In [ ]:

my_figsize = [12, 6]
my_fontsize = 35
WIDTH = 0.03  # the width of the bars
hatches = ["/", "", "*", "\\", "x", "...", "o"]
colors = ["white", "white", "white", "white", "white", "white", "white"]
SCHEMES_label = SCHEMES

def slowdown_bar_plot(mode, labels, workload, flow_range='all'):
    results = {}
    for workload in WORKLOADS:
        results[workload] = []
        for mss in msses:
            for bppn in bppns:
                if flow_range == 'Mice':
                    idx = data[mss][bppn][workload]['SHORT_IDX']
                elif flow_range == 'Elephant':
                    idx = data[mss][bppn][workload]['LONG_IDX']
                elif flow_range == 'middle':
                    idx = data[mss][bppn][workload]['MIDDLE_IDX']
                elif flow_range == 'all':
                    idx = np.arange(len(data[mss][bppn][workload]['FCT']))

                available_idx = data[mss][bppn][workload]['FCT'][:, 7] > 0 # 过滤未完成的流
                idx = idx * available_idx

                fct = data[mss][bppn][workload]['FCT'][idx, 7] # fct

                
                if mode == 'Mean':
                    results[workload].append(np.mean(fct) * 1e3)
                elif mode == 'Median':
                    results[workload].append(np.percentile(fct, 50) * 1e3)
                elif mode == '99p':
                    results[workload].append(np.percentile(fct, 99) * 1e3)



    print(results)


    # SCHEMES_label = SCHEMES
    # SCHEMES_label = ['NegotiaToR(1x)', 'Benes-VLB(1x)', 'Benes-VLB(1.5x)', 'Benes-VLB(2x)']
    # SCHEMES_label = ['NegotiaToR', 'PIM']
    # SCHEMES_label = ['multi_dst_grant', 'single_dst_grant']
    # SCHEMES_label = ['100G', '200G', '300G', '400G']
    # SCHEMES_label = ['NegotiaToR(1x)', 'NegotiaToR(2x)', 'NegotiaToR(3x)', 'NegotiaToR(4x)', 'NegotiaToR(5x)']
    # SCHEMES_label = ['Sirius(1x)', 'Sirius(2x)', 'Sirius(3x)', 'Sirius(4x)', 'Sirius(5x)']
    # SCHEMES_label = ['NegotiaToR(old)', 'NegotiaToR(new)', 'NegotiaToR(2x)', 'VLB(2x)']
    # SCHEMES_label = ['1-iterative', '3-iterative', '5-iterative', '7-iterative']
    # SCHEMES_label = ['ITER_1', 'ITER_2', 'ITER_3', 'ITER_4', 'ITER_5', 'ITER_6', 'ITER_7']
    # SCHEMES_label = ['VLB(1x)', 'VLB(2x)', 'VLB(3x)', 'VLB(4x)', 'VLB(5x)']


    for pias in PIAS:
        x_tick = np.arange(len(labels))*0.3 # the label locations
        x = x_tick - (len(SCHEMES)-1)*WIDTH/2

        fig, ax = plt.subplots(figsize=my_figsize)
        plt.subplots_adjust(left=0.16, right=0.96, bottom=0.15, top=0.9)

        ax.spines['bottom'].set_linewidth('2.0')
        ax.spines['top'].set_linewidth('2.0')
        ax.spines['left'].set_linewidth('2.0')
        ax.spines['right'].set_linewidth('2.0')
        
        ax.grid(axis='y')
        
        for i, scheme in enumerate(SCHEMES):
            # ax.bar(x + WIDTH*i, results[scheme][mode], WIDTH, label=scheme)
            # 一次画一个scheme的所有load的bar
            bound = len(SCHEMES_label) / 2
            bound = 4
            if i < len(SCHEMES) - 1:
                ax.bar(x + WIDTH*i, results[scheme][pias], WIDTH, edgecolor="firebrick", label=SCHEMES_label[i], linewidth = 3, color=[colors[i]], hatch=hatches[i])
            else:
                ax.bar(x + WIDTH*i, results[scheme][pias], WIDTH, edgecolor="darkblue", label=SCHEMES_label[i], linewidth = 3, color=[colors[i]], hatch=hatches[i])
            # ax.bar(x + WIDTH*i, results[scheme][pias], WIDTH, edgecolor="firebrick", label=SCHEMES_label[i], linewidth = 3, color=[colors[i]], hatch=hatches[i])




        

        if mode == "Mean":
            # ax.set_ylabel('Avg. FCT of ' + flow_range + " (ms)", fontsize=my_fontsize)
            ax.set_ylabel('Avg. FCT (ms)', fontsize=my_fontsize)
        elif mode == "99p":
            ax.set_ylabel('99-p. FCT (ms)', fontsize=my_fontsize)
        else:
            ax.set_ylabel('FCT of ' + flow_range + " (ms)", fontsize=my_fontsize)
        # ax.set_xlabel("Incast Degree", fontsize=my_fontsize)
        # ax.set_title('Accelerate by adding line rate', fontsize=my_fontsize)
        ax.set_xlabel("Load (%)", fontsize=my_fontsize)
        ax.set_xticks(x_tick)
        # ax.set_yscale("log")
        # ax.set_ylim(10**-2, 10**1)

        plt.xticks(fontsize=my_fontsize-5)
        plt.yticks(fontsize=my_fontsize-5)


        ax.set_xticklabels(labels, fontsize=my_fontsize)
        ax.tick_params(axis='both', which='both', length=5)
        plt.subplots_adjust(left=0.16, right=0.96, bottom=0.15, top=0.9)


        # scientific_formatter = FuncFormatter(scientific)
        # ax.yaxis.set_major_formatter(scientific_formatter)
        # ax.yaxis.set_minor_formatter(mticker.ScalarFormatter())

        # ax.set_yscale('log')
        # ax.ticklabel_format( style='plain', axis='y')

        # ax.set_yscale('log')

        # ax.set_ylim([0.0001,1000])
        ax.legend(ncol=1, loc = "upper left", fontsize=my_fontsize-20, framealpha=0.3)
        # ax.set_title("Thin-Clos Topology", fontsize=my_fontsize)
        # ax.set_yscale('log')
        # ax.legend(ncol=2, loc = "upper left", fontsize=my_fontsize-2, handlelength=1.5, borderpad = 0.2, labelspacing = 0.3, columnspacing = 1.0)

        # figure_path = '../FIGS/'+ WORKLOADS[0] + '/comparison_fct_' + flow_range + '.pdf'
        figure_path = '../FIGS/NegotiaToR Figs/4/comparison_fct_' + flow_range + '.pdf'
        # plt.savefig(figure_path, dpi=300, bbox_inches='tight')

# ax = plt.subplot(grid[0, 0])
slowdown_bar_plot("Mean", labels, WORKLOADS[0], flow_range='all')

# ax = plt.subplot(grid[1, 0])
slowdown_bar_plot("Mean", labels, WORKLOADS[0], flow_range='Elephant')
slowdown_bar_plot("99p", labels, WORKLOADS[0], flow_range='Mice')